# --- Day 8: Seven Segment Search --- 

https://adventofcode.com/2021/day/8

## Get Input Data

In [39]:
def parse_data(filename):
    with open(f'../inputs/{filename}') as file:
        data = []
        for line in file:
            data.append([x.split(' ') for x in line.strip().split(' | ')])
    return data

In [50]:
test_data = parse_data('test-day-8.txt')
test_data[:2]

[[['be',
   'cfbegad',
   'cbdgef',
   'fgaecd',
   'cgeb',
   'fdcge',
   'agebfd',
   'fecdb',
   'fabcd',
   'edb'],
  ['fdgacbe', 'cefdb', 'cefbgd', 'gcbe']],
 [['edbfga',
   'begcd',
   'cbg',
   'gc',
   'gcadebf',
   'fbgde',
   'acbgfd',
   'abcde',
   'gfcbed',
   'gfec'],
  ['fcgedb', 'cgb', 'dgebacf', 'gc']]]

In [49]:
data = parse_data('day-8.txt')
data[:2]

[[['ecdbfag',
   'deacfb',
   'acdgb',
   'cdg',
   'acdbf',
   'gdfb',
   'efacdg',
   'gd',
   'cagdbf',
   'beacg'],
  ['cdg', 'dcebgaf', 'gbdf', 'bdacg']],
 [['fadecg',
   'gdbecaf',
   'agbfd',
   'fgdcb',
   'gab',
   'ebagdf',
   'feabcg',
   'deab',
   'gdefa',
   'ab'],
  ['adfbg', 'ab', 'fcgdbae', 'bfgecda']]]

## Part 1
---

In [46]:
def count_easy_digits(data):
    """Count easy in the output data."""
    
    num_easy_digits = 0
    
    for row in data:
        for d in row[1]:
            if len(d) in [2, 3, 4, 7]:
                num_easy_digits += 1
    
    return num_easy_digits

### Run on Test Data

In [47]:
count_easy_digits(test_data)  # Should return 26

26

### Run on Input Data

In [48]:
count_easy_digits(data)

525

## Part 2
---

In [ ]:
top
top_left
top_right
middle
bottom_left
bottom_right
bottom

### Run on Test Data

### Run on Input Data